# Projet 7 : Implémentez un modèle de scoring

<hr style="border:2px solid black"> </hr>

Vous êtes Data Scientist au sein d'une société financière, nommée "Prêt à dépenser",  qui propose des crédits à la consommation pour des personnes ayant peu ou pas du tout d'historique de prêt.

L’entreprise souhaite **mettre en œuvre un outil de “scoring crédit” pour calculer la probabilité qu’un client rembourse ou non son crédit**, **puis classifie la demande en crédit accordé ou refusé**. Elle souhaite donc développer un algorithme de classification en s’appuyant sur des sources de données variées (données comportementales, données provenant d'autres institutions financières, etc.).

De plus, les chargés de relation client ont fait remonter le fait que les clients sont de plus en plus demandeurs de transparence vis-à-vis des décisions d’octroi de crédit. Cette demande de transparence des clients va tout à fait dans le sens des valeurs que l’entreprise veut incarner.

**Prêt à dépenser décide donc de développer un dashboard interactif pour que les chargés de relation client puissent à la fois expliquer de façon la plus transparente possible les décisions d’octroi de crédit, mais également permettre à leurs clients de disposer de leurs informations personnelles et de les explorer facilement.**

<hr style="border:0.1px solid black"> </hr>

## Sommaire

- Mise en place de l'environnement virtuel
- Modélisation
 - Cadre de la modélisation
 - Choix du meilleur modèle de classification
   - Régression Logistique
   - SVM
   - Forêt Aléatoire
  - Mise en oeuvre du modèle et enregistrement des résultats
  - Features importance

<hr style="border:0.1px solid black"> </hr>

## Mise en place de l'environnement virtuel

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from pprint import pprint
from time import time
from collections import Counter

In [3]:
from sklearn import metrics,model_selection,preprocessing,linear_model,svm
from sklearn.model_selection import train_test_split,GridSearchCV,ShuffleSplit
from sklearn.metrics import make_scorer, accuracy_score, recall_score,precision_score, recall_score, f1_score, fbeta_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

In [4]:
import pickle
import shap

<hr style="border:0.1px solid black"> </hr>

## Modélisation

On se situe ici dans un **problème d'apprentissage de classification supervisée** ; l'objectif est d'utiliser un ensemble de données financières et socio-économiques pour **prédire si un demandeur sera en mesure de rembourser un prêt ou non**
- **supervisée** : les valeurs cibles à prédire sont "déjà" incluses dans nos données d'apprentissage et l'objectif est de former un modèle pour apprendre à prédire ces valeurs cibles à partir des différents features
- **classification** : la valeur cible est une variable binaire ; 0 signifie que le client remboursera le prêt à temps et 1 signifie que le client aura au contraire des difficultés à rembourser le prêt (ici on parle de **classification binaire** : il s'agit en fait de distinguer si un prêt appartient ou non à une classe)

### Cadre de la modélisation

*Données utilisées*

In [5]:
df = pd.read_csv('C:\\Users\\pauline_castoriadis\\Documents\\implement_scoring_model\\data\\df_train.csv', sep=',')
df.drop("Unnamed: 0", axis = 1, inplace = True)

In [6]:
df.shape

(307511, 27)

*Problèmatique du déséquilibre*

On se rappelle ici qu'on se trouve face à **un problème déséquilibré** : en effet (et heureusement), il **existe davantage de crédits remboursés (valeur 0) que de crédits non remboursés (valeur 1)** ; notre objectif reste bien de **classifier au plus juste ces clients qui ont une forte probabilité de ne pas rembourser**

In [7]:
df['target'].value_counts()

0.0    282686
1.0     24825
Name: target, dtype: int64

La plupart des algorithmes d'apprentissage automatique fonctionnent mieux **lorsque le nombre d'échantillons dans chaque classe est à peu près égal** : cela s'explique par le fait que la plupart des algorithmes sont conçus pour maximiser la précision et réduire les erreurs

**Plusieurs options s'offrent à nous** : 
- Collecter plus de données sur la classe minoritaire
- Réduire le nombre d’individus dans la classe majoritaire
- Dupliquer des individus sous-représentés
- Choix d’une métrique de performance adaptée (on sait qu'on veut traiter plus préciser des faux négatifs)
- Création d’individus « synthétiques »
- Pondération des observations dans le training

*Préparation des données*

Notre base de données, ainsi nettoyée et explorée dans le précédent notebook, contient près de **350k demandes de prêts** et une **trentaine de variables**, dont notre variable cible (la **variable "target"**)

In [8]:
cible = 'target'

En mettant de côté la variable id ("applicant_loan_id") et la variable qui nous permettra de séparer les sets de données ("type_of_set"), on rappelle qu'on a donc mis en place **27 variables prédictrices**

In [9]:
variables_prédictrices =  list(df.loc[:, (df.columns != 'applicant_loan_id') & (df.columns != 'type_of_set') & (df.columns != cible)])

*Choix de la meilleure metric d'évaluation*

Afin d'évaluer la performance de notre classification, nous avons plusieurs outils à notre disposition :
- **Matrice de confusion** : en fonction des valeurs prises par nos classes **réelles** et nos classes **prédites**, on peut calculer le nombre de **faux positifs** et de **faux négatifs** puis de **vrais positifs** et de **vrais négatifs**
- **Rapport de classification** : permet de mettre en évidence les indicateurs suivants
  - **Rappel/sensibilité** (recall/sensitivity): **taux de vrais positifs** ; il s'agit de la proportion de positifs que l’on a correctement identifié (capacité de notre modèle à identifier tous les gens qui ne seraient pas capables de rembourser leur prêt)
  - **Précision** (accuracy) : proportion de prédictions correctes parmi les points que l’on a prédits positifs
  - **F-mesure** : **moyenne harmonique du rappel et de la précision**
  - **Spécificité** (specificity) : **taux de vrais négatifs** (mesure complémentaire de la sensibilité)
- **Courbe ROC** : par une courbe, permet de comprendre comment la sensibilité évolue en fonction de la spécificité (pour chaque seuil de décision possible)

Nous allons **mettre en place le f beta score, qui permet d'identifier un bon compromis entre précision et recall** (sachant qu'il n'existe pas de taux précis pour identifier les faux négatifs, il faut trouver un moyen détourné)

In [10]:
score = make_scorer(fbeta_score, beta = 2.8)

*Séparation jeu d'entrainement et jeu de test*

In [11]:
df = df.sample(n = 10000) # Echantillon pour faire tourner jusqu'au bout le modèle

On procède à une **séparation de nos données au sein de notre jeu de données d'entrainement pour obtenir la validation de la bonne performance de notre modèle de machine learning** (en plus du fait que les étiquettes attribuées à chaque client sont déjà présentes uniquement dans le jeu d'entrainement)

In [12]:
X = df[variables_prédictrices].values
y = df[cible].values

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [14]:
sm = SMOTE(random_state = 42)
X_train, y_train = sm.fit_resample(X_train, y_train)

In [15]:
print(Counter(y_train))

Counter({0.0: 7326, 1.0: 7326})


*Mise à l'échelle des données*

En plus de séparer nos données en données d'entrainement et de test, il faut ramener à une échelle similaire nos différentes données (qui sont de natures extremement différentes) ; **on applique ce traitement uniquement à nos variables prédictrices** (**méthode de standardisation des données**)

In [16]:
std_scale = preprocessing.StandardScaler().fit(X_train)
X_train = std_scale.transform(X_train)
X_test = std_scale.transform(X_test)

### Choix du meilleur modèle de classification

#### Régression Logistique

La régression logistique est un algorithme utilisé pour les problèmes de classification, c'est un algorithme d'analyse prédictive et basé sur le **concept de probabilité**. Nous pouvons appeler une régression logistique un modèle de régression linéaire mais la régression logistique utilise une fonction de coût plus complexe, cette fonction de coût peut être définie comme la « fonction sigmoïde » ou également connue sous le nom de « fonction logistique » au lieu d'une fonction linéaire

L'hypothèse de régression logistique tend à limiter la fonction de coût entre 0 et 1. Par conséquent, les fonctions linéaires ne parviennent pas à le représenter car il peut avoir une valeur supérieure à 1 ou inférieure à 0, ce qui n'est pas possible selon l'hypothèse de la régression logistique

In [17]:
lr_0 = LogisticRegression(random_state = 42)
pprint(lr_0.get_params())

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 100,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}


Nous allons régler les hyperparamètres suivants :
- **C** = Inverse de la force de régularisation (des valeurs plus petites indiquent une régularisation plus forte)
- **solver** = Algorithme à utiliser pour le problème d'optimisation. Pour les problèmes multi-classes, seuls newton-cg, sag, saga et lbfgs gèrent la perte multinomiale
- **class_weight** = Poids associés aux classes (pour traiter en amont le potentiel problème de déséquilibre)
- **penalty** = Utilisé pour spécifier la norme utilisée dans la pénalisation

In [18]:
C = [100, 10, 1.0, 0.5, 0.1, 0.01]
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
class_weight = ['balanced']
penalty = ['none', 'l1', 'l2', 'elasticnet']

In [19]:
lr_param_grid = {'C': C,'solver': solver,'class_weight': class_weight,'penalty': penalty}

In [20]:
lr_model = LogisticRegression(random_state = 42)

Pour l'instant, on réalise simplement la mesure de la performance de notre modèle dans sa capacité à prédire la bonne catégorie binaire (0 ou 1 ici simplement)

In [21]:
def set_model_perf (X_train,X_test,y_train,y_test,modèle,param_grid,score,titre):
    """
    Recherche les meilleurs hyperparamètres
    Entraine et analyse les performances du modèle à partir du jeu d'entrainement
    """
    # Application sur le modèle
    clf = model_selection.GridSearchCV(modèle,param_grid,cv = 10,scoring = score)
    clf.fit(X_train, y_train)
    
    # Afficher le(s) hyperparamètre(s) optimaux
    print("\nMeilleur(s) hyperparamètre(s) sur le jeu d'entraînement:")
    print(clf.best_params_)

    # Mise en oeuvre du modèle avec les meilleurs paramètres
    clf = clf.best_estimator_
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Calculs des indicateurs permettant de qualifier la qualité du modèle, sur le jeu d'entrainement
    temps_calcul = end - start
    y_pred = clf.predict(X_train)
    fbeta = fbeta_score(y_train, y_pred, average = 'weighted', beta = 0.5)
    recall = recall_score(y_train, y_pred, average = 'weighted')
    precision = precision_score(y_train, y_pred, average = 'weighted')
    
    # Mise en forme sous format dataframe
    df_indicateurs = pd.DataFrame()
    
    # Résultats sous format dataframe
    df_indicateurs.loc[1, "Modèle"] = titre
    df_indicateurs.loc[1, "temps calcul"] = temps_calcul
    df_indicateurs.loc[1, "fbeta_score"] = round(fbeta, 2)
    df_indicateurs.loc[1, "recall"] = round(recall, 2)
    df_indicateurs.loc[1, "precision"] = round(precision, 2)
    
    return df_indicateurs.reset_index().drop("index", axis = 1)

In [22]:
table_lr = set_model_perf(X_train,X_test,y_train,y_test,lr_model,lr_param_grid,score,'Régression logistique')


Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'C': 0.1, 'class_weight': 'balanced', 'penalty': 'l1', 'solver': 'liblinear'}


In [23]:
table_lr

,Modèle,temps calcul,fbeta_score,recall,precision
0,Régression logistique,0.039444,0.66,0.66,0.66


#### SVM

Les algorithmes SVM sont :
- Utilisés pour des problèmes de classification
- Généralement utilisé pour un ensemble de données plus petit car il prend du temps à traiter les données
- **C'est l'idée de trouver un hyperplan qui sépare au mieux les caractéristiques en différents domaines**

In [24]:
svc_0 = svm.SVC(random_state = 42)
print('Hyperparamètres possibles et leurs valeurs par défaut:\n')
pprint(svc_0.get_params())

Hyperparamètres possibles et leurs valeurs par défaut:

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': 42,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}


Nous allons régler les hyperparamètres suivants :
- **C** = Paramètre de pénalité C du terme d'erreur
- **kernel** = Spécifie le type de noyau à utiliser dans l'algorithme
- **class_weight** = Même problématique que précédemment
- **gamma** = Coefficient du noyau

In [25]:
C = [.001, .01]
gamma = [.01, .1, 1, 10]
class_weight = ['balanced']
kernel = ['linear', 'rbf', 'poly']

In [26]:
svm_param_grid = {'C': C,'kernel': kernel,'gamma': gamma,'class_weight': class_weight}

In [27]:
svm_model = svm.SVC(random_state = 42)

In [28]:
table_svm = set_model_perf(X_train,X_test,y_train,y_test,svm_model,svm_param_grid,score,'SVM')


Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'C': 0.01, 'class_weight': 'balanced', 'gamma': 1, 'kernel': 'poly'}


In [29]:
table_svm

,Modèle,temps calcul,fbeta_score,recall,precision
0,SVM,159.380898,0.97,0.97,0.97


#### Forêt Aléatoire

La forêt aléatoire se base sur un grand nombre d'abres de décision individuels qui fonctionnent comme un ensemble (il s'agit d'un modèle de la famille des algorithmes ensemblistes, mais qui fonctionnent en parallèle et non pas de manière séquentielle via les apprenants faibles)

In [30]:
rf_0 = RandomForestClassifier(random_state = 42)
print('Hyperparamètres possibles et leurs valeurs par défaut:\n')
pprint(rf_0.get_params())

Hyperparamètres possibles et leurs valeurs par défaut:

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 42,
 'verbose': 0,
 'warm_start': False}


Nous allons régler les hyperparamètres suivants :
- **n_estimators** = nombre d'arbres dans la forêt (nombre d'apprenants faibles)
- **max_depth** = nombre maximum de niveaux dans chaque arbre de décision (profondeur de chaque arbre)
- **bootstrap** = méthode d'échantillonnage des points de données (avec ou sans remplacement)
- **class_weight** = même chose que précédemment

In [31]:
n_estimators = [100,500,1000]
class_weight = ['balanced']
max_depth = [3,5,10,15]
bootstrap = [True]

In [32]:
rf_param_grid = {'n_estimators': n_estimators,'max_depth': max_depth,'bootstrap': bootstrap,
                 'class_weight' : class_weight}

In [33]:
rf_model = RandomForestClassifier(random_state = 42)

In [34]:
table_rf = set_model_perf(X_train,X_test,y_train,y_test,rf_model,rf_param_grid,score,'Forêt Aléatoire')


Meilleur(s) hyperparamètre(s) sur le jeu d'entraînement:
{'bootstrap': True, 'class_weight': 'balanced', 'max_depth': 15, 'n_estimators': 1000}


In [35]:
table_rf

,Modèle,temps calcul,fbeta_score,recall,precision
0,Forêt Aléatoire,70.36445,0.99,0.99,0.99


### Mise en oeuvre du modèle et enregistrement des résultats

In [36]:
frames = [table_lr, table_svm, table_rf]
result = pd.concat(frames)
result

,Modèle,temps calcul,fbeta_score,recall,precision
0,Régression logistique,0.039444,0.66,0.66,0.66
0,SVM,159.380898,0.97,0.97,0.97
0,Forêt Aléatoire,70.364450,0.99,0.99,0.99


Basé sur la performance du fbeta_score, on sélectionne **le meilleur modèle pour ce problème**, ainsi que ses **meilleurs hyperparamètres** (la meilleure combinaison)

In [38]:
best_model = RandomForestClassifier(n_estimators = 1000,max_depth = 15 ,bootstrap = 'True' ,class_weight = 'balanced')

In [39]:
best_model.fit(X_train,y_train)
predict_train = best_model.predict(X_train)
fbeta = fbeta_score(y_train, predict_train, average = 'weighted', beta = 0.5)
print(fbeta)

0.9895899367272645


In [40]:
best_model.fit(X_test,y_test)
predict_test = best_model.predict(X_test)
fbeta = fbeta_score(y_test, predict_test, average = 'weighted', beta = 0.5)
print(fbeta)

1.0


In [41]:
prediction = best_model.predict_proba(X)
prediction

array([[0.86452889, 0.13547111],
       [0.83443777, 0.16556223],
       [0.861     , 0.139     ],
       ...,
       [0.85835916, 0.14164084],
       [0.86209864, 0.13790136],
       [0.84467593, 0.15532407]])

In [43]:
with open('C:\\Users\\pauline_castoriadis\\Documents\\implement_scoring_model\\model\\best_model.pkl', 'wb') as files:
    pickle.dump(best_model, files)

### Pertinence des features utilisés

#### Features importance

En plus d'obtenir d'obtenir un modèle précis, on veut avoir un **modèle interprétable**, c'est à dire être en mesure d'**identifier les variables les plus importantes** (ie. les caractéristiques les plus importantes pour expliquer la variable cible)

In [44]:
importance = pd.DataFrame()
importance["Feature"] = variables_prédictrices
importance["Poids"] = best_model.feature_importances_
importance.sort_values("Poids", ascending = False).head(10)

,Feature,Poids
17,bureau_seniority_past_loans,0.087814
12,applicant_age,0.084926
13,annuity_share_to_income,0.084655
22,previous_application_accepted_share,0.082530
23,previous_application_credit_term,0.077866
4,total_credit_amount,0.072737
8,level_pop_living_region,0.070891
3,applicant_total_income,0.058305
9,applicant_occupation,0.047865
15,bureau_count_past_loans,0.045882


On peut ainsi observer que l'âge du demandeur de prêt a une forte importance dans notre modèle ; ces informations nous seront très utiles pour savoir quelles variables mettre en avant dans notre dashboard streamlit

#### Valeurs de Shapley

Calculer ces valeurs présente trois avantages principaux :
- Le premier est **l'interprétabilité globale** - les valeurs SHAP collectives peuvent montrer **dans quelle mesure chaque prédicteur contribue, positivement ou négativement, à la variable cible**
- Le deuxième avantage est **l'interprétabilité locale** - chaque observation obtient son propre ensemble de valeurs SHAP (on peut expliquer pourquoi un individu reçoit sa prédiction et les contributions des prédicteurs)
- Troisièmement, **les valeurs SHAP peuvent être calculées pour n'importe quel modèle basé sur un arbre**, tandis que d'autres méthodes utilisent des modèles de régression linéaire ou de régression logistique comme modèles de substitution

*Interprétabilité globale*

In [ ]:
shap_values = shap.TreeExplainer(best_model).shap_values(X_train)
shap.summary_plot(shap_values, X_train, plot_type="bar")

Ici on peut lire les informations suivantes :
- **Importance des caractéristiques** : les variables sont classées par ordre décroissant (les plus importantes pour nous se situent en haut)
- **Impact** : l'emplacement horizontal indique si l'effet de cette valeur est associé à une prédiction supérieure ou inférieure 

*Interprétabilité locale*

In [ ]:
def local_shap_interpretability(values,individual):
    """"
    Permet de donner par individu sélectionné l'interprétabilité locale des valeurs de shapley
    """
    data = values[individual]
    array = data.reshape(1, -1)
    explainer = shap.TreeExplainer(best_model)
    shap_values = explainer.shap_values(data)
    shap.initjs()
    return shap.force_plot(explainer.expected_value[individual], shap_values[individual], data)

In [ ]:
local_shap_interpretability(X,1)